In [1]:
import tweepy
import plotly
import datetime

import matplotlib as plt
import pandas as pd
import matplotlib.pyplot as plt 
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots
from datetime import timedelta

In [9]:
# Keys
access_token = '1258532548134797313-w24ae6Fc7uIHO0jXk40akPLrFfUvMW'

access_token_secret = 'L8PwULD3RHMYHnxc7cJlvZSYtXTQORc0ElEFIfL1S7V21'

consumer_key = 'YuJdN38YPbdwFJpAUooTHhGbh'

consumer_key_secret = 'IOzsqk0Vly9Y90xhXqXQqQ1tsyLS11zgVTjGnpdMTkYJzKH2Dl'

# Start the API 
auth = tweepy.OAuthHandler(consumer_key,consumer_key_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True, compression=True)

In [24]:
# Grab NB users
GG_followers = api.followers_ids('GGallegos24')

followers_list = []

# Grab NB followers and their data
for f in tweepy.Cursor(api.followers,screen_name = 'GGallegos24').items():
    try:
        # Create dictionary for specific follower and save data in there
        follower = {}
        follower['id'] = f.id
        follower['created_at'] = f.created_at
        follower['screen_name'] = str(f.screen_name)
        follower['description'] = str(f.description)
        follower['follower_count'] = int(f.followers_count)
        follower['following_count'] = int(f.friends_count)
        follower['favorites_count'] = int(f.favourites_count)
        follower['tweet_count'] = int(f.statuses_count)

    except:
        pass
    # Append user data to list
    followers_list.append(follower)


Rate limit reached. Sleeping for: 894


In [16]:
# Create DataFrame and view head()
followers_df = pd.DataFrame(followers_list)
print(followers_df.head(5))

id          created_at      screen_name  \
0  1138142884857671682 2019-06-10 17:56:24  CarlosE05389219   
1  1274749535437434883 2020-06-21 17:02:40   Gladys83191712   
2  1144452045912457216 2019-06-28 03:46:45   Flores71398267   
3  1240516725814288384 2020-03-19 05:53:40     Melissa56788   
4  1274806607885430784 2020-06-21 20:49:33  William06148787   

            description  follower_count  following_count  favorites_count  \
0      Fun and original               3               26              646   
1      La vida es bella               1               15                1   
2  Aprendiendo cada dia               1              106              143   
3                                     0               36               88   
4                  Walk               1              102                0   

   tweet_count  
0          152  
1            0  
2           30  
3            0  
4            0  


In [18]:
# Create extra columns for analysis

## Count numbers in ScreenName for each follower
followers_df['count_numbers_screen_name'] = followers_df['screen_name'].str.count('[0-9]')
followers_df['count_numbers_screen_name'].apply(int)

## Determine if description is empty
def description_length(description):
    if len(description) == 0:
        return "W/ Description"
    else:
        return "W/O Description"

followers_df['description_empty'] = followers_df['description'].apply(description_length)

## Count number of words in description
followers_df['description_character_count'] = followers_df['description'].apply(len)

## Calculate account age
def calculate_age_days(created_at):
    delta = datetime.datetime.now() - created_at 
    return(int(delta.days))

def calculate_age_hours(created_at):
    delta = datetime.datetime.now() - created_at 
    delta = (delta.days*24) + (delta.seconds//3600)
    return(int(delta))

followers_df['account_age_days'] = followers_df['created_at'].apply(calculate_age_days)
followers_df['account_age_hours'] = followers_df['created_at'].apply(calculate_age_hours)

## Numbers in Name >= 4
def numbers_name(count):
    if count >=5:
        return(1)
    else:
        return(0)
followers_df['numbers_in_name_5'] = followers_df['count_numbers_screen_name'].apply(numbers_name)

print(followers_df.head(5))

id          created_at      screen_name  \
0  1138142884857671682 2019-06-10 17:56:24  CarlosE05389219   
1  1274749535437434883 2020-06-21 17:02:40   Gladys83191712   
2  1144452045912457216 2019-06-28 03:46:45   Flores71398267   
3  1240516725814288384 2020-03-19 05:53:40     Melissa56788   
4  1274806607885430784 2020-06-21 20:49:33  William06148787   

            description  follower_count  following_count  favorites_count  \
0      Fun and original               3               26              646   
1      La vida es bella               1               15                1   
2  Aprendiendo cada dia               1              106              143   
3                                     0               36               88   
4                  Walk               1              102                0   

   tweet_count  count_numbers_screen_name description_empty  \
0          152                          8   W/O Description   
1            0                          8   W/O Desc

In [19]:
# Plots
## Following vs Followers
fig = px.scatter(followers_df, x = 'follower_count', y = 'following_count', color = 'account_age_days')

fig.update_layout(
    title = 'Following vs Followers',
    xaxis_title = 'Followers',
    yaxis_title = 'Following'
)

fig.show()

In [20]:
## Following vs Followers
fig = px.scatter(followers_df, x = 'follower_count', y = 'following_count',color = 'account_age_days',facet_col='description_empty',log_x=True,log_y=True)

fig.update_traces(
    marker = dict(size = 8,line = dict(width=1,color='DarkSlateGrey'))
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

fig.update_layout(
    title = 'Following vs Followers',
    xaxis_title = 'Followers',
    yaxis_title = 'Following',
    width = 1000,
    height = 500
)

fig.show()

In [21]:
# Numbers in Screen Name

fig = make_subplots(rows = 2,cols = 2)
## Followers
fig.add_trace(
    go.Scatter(
        x = followers_df['account_age_days'],
        y = followers_df['follower_count'],
        mode = 'markers',
        marker = dict(color = followers_df['account_age_days'])
    ),
        row = 1,
        col = 1
)
## Following
fig.add_trace(
    go.Scatter(
        x = followers_df['account_age_days'],
        y = followers_df['following_count'],
        mode = 'markers',
        marker = dict(color = followers_df['account_age_days'])
    ),
        row = 1,
        col = 2
)

## Tweets
fig.add_trace(
    go.Scatter(
        x = followers_df['account_age_days'],
        y = followers_df['tweet_count'],
        mode = 'markers',
        marker = dict(color = followers_df['account_age_days'])
    ),
        row = 2,
        col = 1
)

## Favorites
fig.add_trace(
    go.Scatter(
        x = followers_df['account_age_days'],
        y = followers_df['favorites_count'],
        mode = 'markers',
        marker = dict(color = followers_df['account_age_days'])
    ),
        row = 2,
        col = 2
)

## Update marker 
fig.update_traces(
    marker = dict(size = 8,line = dict(width=1,color='DarkSlateGrey'))
)

## Update xaxis labels
fig.update_yaxes(title='Followers Count', row = 1, col = 1)
fig.update_yaxes(title='Following Count', row = 1, col = 2)
fig.update_yaxes(title='Tweets Count', row = 2, col = 1)
fig.update_yaxes(title='Favorites Count', row = 2, col = 2)

## Update Title
fig.update_layout(
    title_text = 'Activity vs Account Age (Days)',
    width = 1000,
    height = 700,
    showlegend = False)

fig.show()

In [22]:
# Count by Account Age in Hours
df_mean =followers_df.groupby('account_age_hours').count().reset_index()

In [23]:
## Histogram
fig = px.histogram(
    df_mean,
    x = 'account_age_hours',
    title = 'Distribution of Followers by Account Age (hours)',
    opacity = 0.5,
    nbins = 25
)

fig.show()